In [1]:
import torch
from torch import nn

import numpy as np

In [2]:
text = ['hey how are you','good i am fine','have a nice day']
# text = open("lm_data/clean.txt").readlines()
print(text)

chars = set(''.join(text))
int2char = dict(enumerate(chars))
char2int = {char: ind for ind,char in int2char.items()}

['hey how are you', 'good i am fine', 'have a nice day']


In [3]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [4]:
# padding the text
for i in range(len(text)):
    while len(text[i]) < maxlen:
        text[i] += ' '

In [5]:
input_seq = []
target_seq = []

for i in range(len(text)):
    # remove the first token

    input_seq.append(text[i][:-1])
    target_seq.append(text[i][-1])
    print("Input Sequence:{}\n Target Sequence:{}".format(
        input_seq[i],
        target_seq[i]
    ))

Input Sequence:hey how are yo
 Target Sequence:u
Input Sequence:good i am fine
 Target Sequence: 
Input Sequence:have a nice da
 Target Sequence:y


In [6]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [7]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence,dict_size,seq_len,batch_size):
    features = np.zeros((batch_size,seq_len,dict_size),dtype = np.float32)

    for i in range(batch_size):
        for u in range(seq_len):
            features[i,u,sequence[i][u]] = 1
    return features

In [8]:
# input_seq_one_hot = one_hot_encode(input_seq,dict_size,seq_len,batch_size)
# input_seq_one_hot = torch.from_numpy(input_seq_one_hot)

input_seq = torch.Tensor(input_seq)
input_seq


tensor([[ 7.,  6., 16., 10.,  7., 13., 14., 10.,  2.,  8.,  6., 10., 16., 13.],
        [ 1., 13., 13.,  3., 10., 12., 10.,  2.,  5., 10., 11., 12.,  9.,  6.],
        [ 7.,  2.,  4.,  6., 10.,  2., 10.,  9., 12., 15.,  6., 10.,  3.,  2.]])

In [9]:
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.size()))

Input shape: torch.Size([3, 14]) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [10]:
# create tn
import tntorch as tn
import torch
torch.set_default_dtype(torch.float64)
t = tn.rand([17]*14 + [17], ranks_tt=10, ranks_tucker=6, requires_grad=True)
t.set_factors('dct', dim=range(14))
# input_seq_one_hot.view(-1,17).size()
# t[input_seq].torch().size()
t


15D TT-Tucker tensor:

 17  17  17  17  17  17  17  17  17  17  17  17  17  17  17
  |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
  6   6   6   6   6   6   6   6   6   6   6   6   6   6   6
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9) (10)(11)(12)(13)(14)
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
1   10  10  10  10  10  10  10  10  10  10  10  10  10  10  1

In [11]:
t[input_seq]

2D TT-Tucker tensor:

     17
  3   |
  |   6
 (0) (1)
 / \ / \
1   10  1

In [12]:
target_seq = torch.Tensor(target_seq)
target_seq.size()
target_seq = target_seq.squeeze(1).long()
target_seq

tensor([ 0, 10, 16])

In [13]:
def softmax(x):
    expx = torch.exp(x-x.max())
    return expx / torch.sum(expx, dim=-1, keepdim=True)
def cross_entropy_loss(probs, y):
    return torch.mean(-torch.log(probs[np.arange(len(probs)), y]))

In [14]:
def loss(t):
    return cross_entropy_loss(softmax(t[input_seq].torch()), target_seq)
tn.optimize(t, loss)

iter: 0       | loss:   2.976760 | total time:    0.0415
iter: 92      | loss:   0.000053 | total time:    0.8763 <- converged (tol=0.0001)


In [15]:
prediction = torch.max(t[input_seq].torch(), dim=1)[1]
for item in prediction:
    print(int2char[item.item()])


u
 
y


In [16]:
import torch

